This workbook requires PyCall, PyPlot, Interact and SymPy.

For the database example the Chinook database needs to be installed in a MySQL database.<br/>
The MySQL daemon must be running and _mysql.connector_ installed on the Python side.<br/>
The SciPy library should be already available.<br/><br/>

# Accessing in Python via Julia

## Graphics using PyPlot

In [3]:
using PyCall, PyPlot

INFO: Loading help data...


In [ ]:
x = linspace(0,2pi,1001);
y = sin(3*x + 4*cos(2*x));
plot(x,y,color="red",linewidth=2.0, linestyle="--")

In [ ]:
z = linspace(0, 3π, 250);
surf(z, z, z.*sin(z).*cos(z)'.*exp(-0.4z))

---

## MySQL database access

[Chinook](https://chinookdatabase.codeplex.com) is a sample database available for SQL Server, Oracle, MySQL, etc.

The Chinook data model represents a digital media store, including tables for artists, albums, media tracks, invoices and customers.<br/>You can see the Chinook data model [here](https://chinookdatabase.codeplex.com/wikipage?title=Chinook_Schema).

- Media related data was created using real data from an iTunes Library. 

- Customer and employee information was manually created using fictitious names, addresses that can be located on Google maps, and other well formatted data (phone, fax, email, etc.).

- Sales information is auto generated using random data for a four year period.


### Query the Chinook database
[Get the database files from here](https://chinookdatabase.codeplex.com)

### Python code

```python
import mysql.connector as mc
cnx = mc.connect(user="malcolm", password="mypasswd", database="Chinook")
csr = cnx.cursor()
qry = "select a.title, b.name from Album a, Artist b where a.ArtistId = b.ArtistId limit 10;"
csr.execute(qry)
for vals in csr:
    print(vals)
    
csr.close()
cnx.close()
```

For this code to work the MySQL daemon must be running and the Chinook data loaded.<br/>If is sensible to test this from the _mysql_ client before executing this code.

The _&lt;user&gt;_ chosen must be granted access rights to the data (at least SELECT privilige ).

THe connection string needs to be amended to reflect the specific: **user/password/database**.  

In [ ]:
# Julia version

@pyimport mysql.connector as mc
cnx = mc.connect(user="malcolm", password="mypasswd", database="Chinook")

In [ ]:
macroexpand(:(@pyimport mysql.connector as mc))

In [ ]:
qry = "select a.title, b.name from Album a, Artist b ";
qry *= "where a.ArtistId = b.ArtistId limit 10;";
csr = cnx[:cursor]();
csr[:execute](qry);

In [ ]:
for vals in csr
  album  = vals[1]
  artist = vals[2]
    @printf "%s by %s\n" album artist
end

In [ ]:
csr[:close]();
cnx[:close]();

---

## Using SciPy

### Nelder-Mead Simplex algorithm
[Description of the problem](http://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html)

The simplex algorithm is probably the simplest way to minimize a fairly well-behaved function. 

It requires only function evaluations and is a good choice for simple minimization problems. 

However, because it does not use any gradient evaluations, it may take longer to find the minimum.

In [ ]:
@pyimport scipy.optimize as so

In [ ]:
x0 = [1.3, 0.7, 0.8, 1.9, 1.2, 1.1];
opts = Dict();
opts["xtol"] = 1e-8; opts["disp"] =  true;

In [ ]:
rosen(x) = sum(100.0*(x[2:end] - x[1:end-1].^2.0).^2.0 + (1 - x[1:end-1]).^2.0);

In [ ]:
so.minimize(rosen, x0, method="nelder-mead",options=opts);

---

### Least Squares Fit

The least square algorithm is a well tried and tested technique.

It minimises the sum of the residuals of a _fitted_ curve with its data points in order to estimate the parameters.

Here we create a sinusoid and add some noise, fit the LSQ curve and compare it with the original and the data points.

In [ ]:
(A, k, θ) = (10.0, 1.0/0.03, π/6.0);
x = collect(0.0:0.002:0.058);
y_true = A * sin(2π*k*x + θ);
y_meas = y_true + 2*randn(length(x));

In [ ]:
function residuals(p, y, x)
  (A, k, θ) = p
  return y - A * sin(2π*k*x + θ)
end;
peval(x, p) =  p[1] * sin(2π*p[2]*x + p[3]);

In [ ]:
p0 = [8, 1/2.3e-2, pi/3];
plsq = so.leastsq(residuals, p0, args=(y_meas, x));

In [ ]:
p = plot(x, y_true);
p = plot(x, y_meas, marker="o", linestyle=" ");
p = plot(x, peval(x, plsq[1]), linestyle="--");

p

---

## Interactive Widgets in IJulia

IPython 2.0 introduced interactive widgets, which are basically:

* Javascript widgets (sliders, buttons, etc.)
* A communications protocol for the widgets to talk to the kernel
* A Python interface to create and manipulate these.

Thanks to [Shashi Gowda](https://github.com/shashi/), these are accessible from a Julia interface.

In [ ]:
using Interact

In [ ]:
@manipulate for n in 1:100
    rand(n,n)
end

### Interactive graphics

In [ ]:
using PyPlot

In [ ]:
x = linspace(0,10,1000)
f = figure()
@manipulate for α = 1:0.1:4, β = 1:0.1:4, title="an interactive plot"
    withfig(f) do
        plot(x, cos(α*x + sin(β*x)))
        legend([title])
    end
end

### Automating SymPy

In [ ]:
using SymPy
x = symbols("x")
@manipulate for n=0:20
    latex(SymPy.diff(sin(x^2), x, n))
end

### FX market ticker

_**Run this from a julia command prompt**_

```julia
using PyCall
@pyimport myfinx as fx
fx.run("MSFT")
```